## Loading Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers
import os

In [18]:
BASE_PATH = './data/'
datasetpath = f'{BASE_PATH}/movie.csv'
os.listdir(BASE_PATH)

['movie.csv']

In [4]:
df = pd.read_csv(datasetpath, encoding='utf-8')
df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [29]:
df.loc[0, 'text']

'I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.'

In [5]:
df.describe()

,label
count,40000.000000
mean,0.499525
std,0.500006
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


In [7]:
df.isnull()

,text,label
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
39995,False,False
39996,False,False
39997,False,False
39998,False,False


In [8]:
df.columns

Index(['text', 'label'], dtype='object')

In [10]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [11]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\renua\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
train_x, test_x, y_train, y_test = train_test_split(df.text,df.label, test_size=0.25,random_state=123)

In [13]:
#TF-IDF
tfv = TfidfVectorizer(min_df=3,  max_features=None, strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
                    ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1, stop_words = 'english')

tfv.fit(list(train_x) + list(test_x))
train_x_tfv =  tfv.transform(train_x) 
test_x_tfv = tfv.transform(test_x)

In [14]:
train_x.shape

(30000,)

In [15]:
y_train.shape

(30000,)

In [16]:
test_x.shape, y_test.shape

((10000,), (10000,))

In [26]:
dataset = tf.data.experimental.make_csv_dataset(datasetpath, batch_size=32,
                                                select_columns=['\ufefftext', 'label'],
                                                label_name='label')

<TakeDataset element_spec=(OrderedDict([('\ufefftext', TensorSpec(shape=(32,), dtype=tf.string, name=None))]), TensorSpec(shape=(32,), dtype=tf.int32, name=None))>


In [36]:
for features, labels in dataset.take(1): # first batch
    print(labels)
    for key, value in features.items():
        print("{:20s}: {}".format(key,value.numpy()))

tf.Tensor([1 0 1 0 0 0 0 0 1 1 1 1 0 1 1 0 1 1 1 0 0 0 0 0 1 0 1 1 0 0 0 1], shape=(32,), dtype=int32)
﻿text               : [b"This movie was recommended to me by the same person that blessed me with a copy of The Chronicles of Narnia. Shadowlands is one of the most amazing screenplays ever written. It is well executed, acted and directed. The cinematography is a bit dark for my taste but I'm sure it was intended to be so. The screenplay is like poetry in portions of the movie, through out the movie I found myself taking pause to reflect on the comments just made on screen. This is a wonderful piece of cinema and I can only hope that more people will run across it and add reviews. Fair warning though this was a 6 tissue movie for me. Very touching. Very Heartfelt performances."
 b'Astounding that something like this could find its way to be viewed by the public. I knew it was by Uwe Boll, & I found it in the bargain bin at a store for $2 (still pretty steep, considering) but morbid cu

In [24]:
dataset = dataset.map(lambda x, y: (tf.concat([tf.expand_dims(x['\ufefftext'], axis=-1)], axis=-1), y))
dataset

TypeError: in user code:

    File "C:\Users\renua\AppData\Local\Temp\ipykernel_17272\4275188195.py", line 1, in None  *
        lambda x, y: (tf.concat([tf.expand_dims(x['\ufefftext'], axis=-1)], axis=-1), y)

    TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got '\ufefftext'


## Building the model

In [21]:
model = tf.keras.Sequential([
    # layers.Input(shape=(30000, )),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [22]:
model.compile('adam', tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [23]:
model.fit(dataset, epochs=10)

Epoch 1/10


UnimplementedError: Graph execution error:

Detected at node 'sequential/dense/Cast' defined at (most recent call last):
    File "C:\Program Files\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "d:\Documents\Mec2a\sentiment-analisys-with-MLP\venv\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "d:\Documents\Mec2a\sentiment-analisys-with-MLP\venv\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "d:\Documents\Mec2a\sentiment-analisys-with-MLP\venv\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "d:\Documents\Mec2a\sentiment-analisys-with-MLP\venv\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    Fil